# TDM for IGWBS 1: Daten von der OAI-Schnittstelle von e-rara

## Womit arbeiten wir hier eigentlich?

In [ ]:
!python --version

In [ ]:
# Path für neu installierte Libraries hinzufügen
import sys
sys.path.append('/home/.local/lib/python3.8/site-packages')

In [ ]:
# vorinstallierte Libraries - bei Interesse "#" vor dem Code entfernen und ausführen
#!pip list

In [ ]:
# Installation fehlender Libraries
!pip install beautifulsoup4 

In [ ]:
# Import der (installierten) Libraries, die im Nachfolgenden benutzt werden

from IPython.display import IFrame              # Webpages in Jupyter Notebook anzeigen/einbetten
import requests                                 # Web-URLs abfragen
from bs4 import BeautifulSoup as soup           # Webscraping und HTML/XML parsen
import lxml                                     # XML Parser für Beautiful Soup
                                                
import os                                       # System und z.B. Ordner navigieren und manipulieren 
import re                                       # mit Regex (Regular Expressions) arbeiten
import pandas as pd                             # Standard Library für Dataframes (tabelleartige Datenstrukturen)
print("Alle Libraries erfolgreich importiert")

## Wie sieht die OAI-Schnittstelle im Web aus?

Die OAI-Schnittstelle von e-rara ist frei zugänglich: ES braucht keine Registrierung oder API Key zur Nutzung der Daten. Diese stehen unter [freien Lizenzen](https://www.e-rara.ch/wiki/termsOfUse) zur Verfügung.

Mit`IFrame` können Websites in einem Jupyter Notebook angezeigt/eingebunden werden, so auch die OAI-Startseite von e-rara, `https://www.e-rara.ch/oai?verb=Identify`.

Probieren Sie verschiedene **OAI verbs** in der OAI-URL im Code unten aus:
* `Identify`
* `ListMetadataFormats`
* `ListSets`.

In [ ]:
# Identify = Startseite der OAI-Schnittstelle
IFrame('https://www.e-rara.ch/oai?verb=Identify', width=820, height=300)

Mit dem OAI verb `GetRecord` und dem entsprechenden **e-rara-Identifier** lassen sich einfach die Metadaten eines bestimmten Titels anzeigen.

In der URL des Titel auf der e-rara-Plattform, z.B. https://www.e-rara.ch/zut/content/titleinfo/26079348, ist der e-rara-Identifier enthalten, im Beispiel: `26079348`.

Probieren Sie verschiedene Metadatenstandards aus. Diese lassen sich mit dem Parameter `metadataPrefix` in der URL einfach variieren:
* `oai_dc` - [Simple Dublin Core](http://purl.org/dc/elements/1.1/)
* `mods` - [MODS](https://www.loc.gov/standards/mods/)
* `mets`- [METS](https://www.loc.gov/standards/mets/)

Und natürlich können Sie auch andere e-rara-Identifier ausprobieren, z.B. `26079348`!

In [ ]:
# Beipieldatensatz aus e-rara
IFrame('https://www.e-rara.ch/oai?verb=GetRecord&metadataPrefix=oai_dc&identifier=20329783', width=820, height=300)

## XML-Daten von der OAI-Schnittstelle beziehen

Bisher haben wir die **Metadaten im  XML-Format** nur auf einer Website angezeigt erhalten - nun greifen wir direkt auf diese Daten zu, d.h. laden sie herunter und speichern sie in Dateien. Hierfür werden zwei kleine Funktionen definiert: `load_xml`und `download_record`. Bei Letzterem wird das OAI verb `GetRecord` benutzt.

In [ ]:
# Basis-URL der OAI-Schnittstelle als definierte Variable macht die weitere Verwendung einfacher
oai = 'https://www.e-rara.ch/oai'

In [ ]:
# allgemeine Funktion zur Datenabfrage der OAI-Schnittstelle und Dekodierung in XML
def load_xml(params):
    '''
    Accesses the OAI interface according to given parameters and scrapes its content.
    Parameters:
    All available native OAI verbs and parameter/value pairs.
    '''
    base_url = oai
    response = requests.get(base_url, params=params)
    output_soup = soup(response.content, features="xml")   #"lxml"
    return output_soup

In [ ]:
# "load_xml"-Funktion benötigt die Parameter, die in den URLs oben angehängt wurden
xml_soup = load_xml({'verb': 'Identify'})
xml_soup

In [ ]:
# OAI verb, metadataPrefix, identifier als Parameter für die "load_xml"-Funktion
# entspricht: https://www.e-rara.ch/oai?verb=GetRecord&metadataPrefix=oai_dc&identifier=26079348

xml_soup = load_xml({'verb': 'GetRecord', 'metadataPrefix': 'oai_dc', 'identifier': '26079348'})
xml_soup

In [ ]:
# Funktion um Metadaten eines Titels herunterzuladen - in verschiedenenen Metadatenstandards und in genuiner XML-Formatierung

def download_record(ID, metadataPrefix='mods'):
    '''
    Downloads a certain metadata record in original XML formatting from OAI to a single XML file.
    Throws a notice when download is successful and if download fails.
    Parameters:
    ID = E-rara ID of the desired metadata record.
    metadataPrefix = Metadata format to be delivered. Can be: oai_dc, mods, rawmods, mets. Default value is mods.
    '''
    path = os.getcwd()
    output_soup = load_xml({'verb': 'GetRecord', 'metadataPrefix': metadataPrefix, 'identifier': ID})
    outfile = path + '/{}.xml'.format(ID) 
    try:
        with open(outfile, mode='w', encoding='utf-8') as f:
            f.write(output_soup.decode())
            print("Metadata file {}.xml saved".format(ID))
    except:
            print("Saving metadata file failed".format(ID))
    finally:
            pass

In [ ]:
# die Funkion in Aktion
download_record('26079348')
#download_record('26079348', metadataPrefix='oai_dc')

In [ ]:
# auch mehrere Titel sind mittels einer FOR-Schleife möglich
for ID in ['22400512', '4310610', '13621365']:
    download_record(ID, metadataPrefix='oai_dc')

## Etwas einfacher: Arbeiten mit der Polymatheia Library

Genuin liefert die OAI-PMH-Schnittstelle XML-formatierte Metadaten gemäss verschiedenen Standards aus. Mit [Polymatheia](https://polymatheia.readthedocs.io/en/latest/index.html), einer spezifischen Python "Library" für OAI-Schnittstellen, können diese nicht nur **einfacher abgefragt** werden, sondern die Daten werden auch in einem **einfacher auswertbaren Format** ausgegeben. Das [Navigable Dictionary](https://polymatheia.readthedocs.io/en/latest/concepts.html) macht die direkte Addressierung einzelner Datenelemente möglich. Zudem kann es leicht im [JSON](https://de.wikipedia.org/wiki/JavaScript_Object_Notation)-Format gespeichert werden.

In [ ]:
# Polymatheia library installieren
!pip install polymatheia

In [ ]:
# verschiedene Module von Polymatheia importieren
from polymatheia.data.reader import OAISetReader               # Fragt die Sets/Sammlungen ab
from polymatheia.data.reader import OAIMetadataFormatReader    # Fragt die Metadatenformate, die angeboten werden, ab
from polymatheia.data.reader import OAIRecordReader            # Fragt mehrere Records/Titeldatensätze ab
from polymatheia.data.writer import PandasDFWriter             # zur Transformation von flachen Daten in Tabellenform (Dataframe)
print("Alle Libraries erfolgreich importiert")

In [ ]:
# Basis-URL der OAI-Schnittstelle als definierte Variable macht die weitere Verwendung einfacher
oai = 'https://www.e-rara.ch/oai'

### Sets/Sammlungen und Metadatenformate abfragen

Polymatheia arbeitet mit vorgefertigten **Readern**, die einfach zur Abfrage gemäss der verschiedenen OAI verbs benutzt werden können. Der Reader nimmt die ausgelieferten Daten entgegen und speichert sie in einem Objekt (einem Navigable Dictionary), das dann ausgelesen werden kann.

In [ ]:
# Der OAIMetadataFormatReader, der das OAI verb "ListMetadataFormats" nutzt
reader = OAIMetadataFormatReader(oai)
type(reader)

In [ ]:
# Welche Metdatenformate sind verfügbar? Daten aus dem Reader auslesen
for formats in reader:
    #print(formats)
    print(formats.metadataPrefix)

In [ ]:
[formats.metadataPrefix for formats in reader]    # abgekürzte Variante der FOR-Schleife oben

In [ ]:
# Welche Sets/Sammlungen bietet die Schnittstelle an? Hierfür gibt es den OAISetReader

reader = OAISetReader(oai)                         
[x for x in reader]

In [ ]:
# Mit dem PandasDFWriter von Polymatheia lassen sich die Set-Daten einfach in eine übersichtlichere Tabellenform überführen

reader = OAISetReader(oai)
oai_sets = []                          # eine leere Liste namens "oai_sets" erstellen
for x in reader:
    oai_sets.append(x)                 # ".append" hängt nacheinander alle Elemente aus "reader" in die Liste "oai_sets"
df = PandasDFWriter().write(oai_sets)  # aus der Liste "oai_sets" den Dataframe "df" erstellen
df.style

### Grösse eines Sets/einer Sammlung abrufen

Von Interesse ist oft auch die Grösse eines Sets/einer Sammlung. Diese ist etwas versteckt in den ausgebenen Daten, z.B. mit dem OAI verb `ListIdentifiers`, vorhanden. Um die Setgrösse einfach abzurufen, wird eine eine weitere Funktion `setsize()` definiert. Das entsprechende Set wird mit seinem Kurzzeichen abgefragt. 

In [ ]:
# Wo steckt die Setgrösse?
IFrame('https://www.e-rara.ch/oai?verb=ListIdentifiers&set=vitruviana&metadataPrefix=oai_dc', width=820, height=300)

In [ ]:
# Wie gross ist ein bestimtmes Set?

def setsize(Set):  
    '''
    Accesses the OAI interface and retrieves the size of a given OAI set.
    Parameters:
    Set: The 'setSpec' short cut of the desired OAI set.
    '''
    base_url = oai
    listsearch_term = {'verb': 'ListIdentifiers', 'metadataPrefix': 'oai_dc', 'set': Set}
    
    # Basic function
    def load_xml(params):
        '''
        Accesses the OAI interface according to given parameters and scrapes its content.
        Parameters:
        All available native OAI verbs and parameter/value pairs.
        '''
        response = requests.get(base_url, params=params)
        output_soup = soup(response.content, "lxml")
        return output_soup
    
    xml_soup = load_xml(listsearch_term)
    if xml_soup.resumptiontoken:
        set_size = int(xml_soup.resumptiontoken['completelistsize'])
    else:
        set_size = len(xml_soup.find_all('identifier'))
    return set_size

In [ ]:
# alle Titel
setsize('')

In [ ]:
# "Gottfried Keller (1819-1890)"
setsize('pbgkeller')

### Auf ausgewählte Metadaten eines Sets zugreifen

Mit Polymatheia lassen sich leicht **massenhaft Metadaten** per OAI herunterladen. Hierbei können Metadatenstandard, Set/Sammlung und Anzahl von Datensätzen definiert werden. Der entsprechende Reader heisst `OAIRecordReader`. Die Daten sind wieder als Navigable Dictionary im Reader-Objekt gespeichert.

In [ ]:
# x Metadatensätze eines Sets/einer Sammlung herunterladen und die "header section" der Datensätze auslesen

reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=2)
for record in reader:
    print(record.header)

In [ ]:
# Einzelne Metadatenfelder können im Navigable Dictionary direkt mit "Subsetting" oder per Punkt-Notation adressiert werden

for record in reader:
    print(record['header']['identifier']['_text'])       # Subsetting aus Python
    print(record.header.identifier._text)                # Punkt-Notation des NavigableDict aus Polymatheia
    print('---')

In [ ]:
# Ebenso kann die "metadata section" des Datensätze ausgelesen werden - übersichtlicher in der abgekürzten Form der FOR-Schleife

[record.metadata for record in reader]

In [ ]:
# Für die Abfrage in der metadata section muss an einer Stelle allerdings zwingend die Subsetting-Syntax verwendet werden 
# wg. geschweiferter Klammern als Sonderzeichen

[record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc'].dc_title._text for record in reader]

In [ ]:
# Versuchen Sie weitere Metadaten-Elemente so herauszulesen!

[record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc']...... for record in reader]

In [ ]:
# Metadatenfelder können auch mehrere Werte enthalten, wie die Angabe von mehreren Sets/Sammlungen, in die ein Titel gehört
# Die Daten werden dann als Liste ausgegeben, zu erkennen an den einfassenden eckigen Klammern.

for record in reader:
    print(record.header.setSpec)

In [ ]:
# Wie beim Reader können die Listen-Elemente mit einer FOR-Schleife einzeln ausgelesen werden.
# D.h. wird haben dann 2 ineinandergeschachtelte FOR-Schleifen

for record in reader:
    for list_item in record.header.setSpec:
        print(list_item._text)
    print('---')

In [ ]:
# Auch das "dc_subject"-Feld in der metadata section enthält eine Liste von mehreren Werten
# Ergänzen Sie den Code, um die einzelnen Werte auszulesen!

for record in reader:
    for list_item in record.metadata['{http://www.openarchives.org/OAI/2.0/oai_dc/}dc']........:
        print(list_item.......)
    print('---')

Es gibt aber noch einen einfacheren Weg, an die einzelnen Metadaten-Elemente heranzuzukommen! Hier gibt es **keine Probleme mehr mit Einzelwerten versus Listen** als Datenwerte. Die einzelnen hierachisch angeordneten Metadaten-Elemente werden als Parameter eines spezifischen Befehls, `get`, gehandhabt.

Zu beachten ist dabei, dass im Falle von Listen ebenso eine Ergebnisliste (in eckigen Klammern) ausgegeben wird.


In [ ]:
# Einfacher Zugriff auf alle Metadaten-Elemente mit dem "get"-Befehl

for record in reader:
    print(record.get(['header', 'identifier', '_text']))
    print(record.get(['header', 'setSpec', '_text']))
    print(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_title', '_text']))
    print(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_subject', '_text']))
    print('---')

### Ausgewählte Metadaten in Tabellenform bringen und als CSV abspeichern

In [ ]:
setsize('ch19')

In [ ]:
reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='ch19', max_records=1000)

identifier = [] 
sets = []
creator = []
title = [] 
year = []
language = []
publisher = []
subjects = []
types = []
rights = []


for record in reader:
    identifier.append(record.get(['header', 'identifier', '_text']))
    sets.append(record.get(['header', 'setSpec', '_text']))
    creator.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_creator', '_text']))
    title.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_title', '_text']))
    year.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_date', '_text']))
    language.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_language', '_text']))
    publisher.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_publisher', '_text']))
    subjects.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_subject', '_text']))
    types.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_type', '_text']))
    rights.append(record.get(['metadata', '{http://www.openarchives.org/OAI/2.0/oai_dc/}dc', 'dc_rights', '_text']))
    
df = pd.DataFrame(list(zip(identifier, title, creator, year, language, publisher, subjects, sets, types, rights)),
               columns =['identifier', 'title', 'creator', 'year', 'language', 'publisher', 'subjects', 'sets', 'types', 'rights'])
df

In [ ]:
# Tabelle als CSV-Datei abspeichern, mit Semikolon als Spaltentrenner
outfile = 'e-rara_daten_semikolon.csv'
with open(outfile, mode='w', encoding='utf-8') as f:
    df.to_csv(f, index=False, sep=';')

In [ ]:
# Tabelle als CSV-Datei abspeichern, mit Komma als Spaltentrenner (default)
outfile = 'e-rara_daten.csv'
with open(outfile, mode='w', encoding='utf-8') as f:
    df.to_csv(f, index=False)

## Ganze Metadatensätze als JSON-Dateien speichern

Daten, die als Navigable Dictionary vorliegen können einfach in **JSON-Formatierung** abgespeichert werden. Hierfür werden die einzelnen Datensätze aus dem Reader gelesen und **mit `.json` Dateiendung** mit dem **e-rara-Identifier als Dateiname** gespeichert. Im Beispiel wird vorab noch ein Ordner `json_data_dc` für die entstehenden Dateien angelegt.

In [ ]:
# Auslesen der e-rara-Identifier eines Sets
reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=10)

for record in reader:
    match = re.search('oai:www.e-rara.ch:(\d+)', record.header.identifier._text)
    ID = match.group(1)
    print(ID)

In [ ]:
# Metadaten eines Sets/einer Sammlung als JSON-Dateien abspeichern, mit e-rara Identifiern als Dateiname
# Versuchen Sie auch das Gleiche im MODS-Standard und vergleichen Sie die abgespeicherten JSON-Dateien!

reader = OAIRecordReader(oai, metadata_prefix='oai_dc', set_spec='stp', max_records=10)
os.makedirs('json_data_dc', exist_ok=True)
path = 'json_data_dc'

for record in reader:
    match = re.search('oai:www.e-rara.ch:(\d+)', record.header.identifier._text)
    ID = match.group(1)
    outfile = path + '/{}.json'.format(ID)
    try:
        with open(outfile, mode='w', encoding='utf-8') as f:
            f.write(str(record))
            print("Metadata file {}.json saved".format(ID))
    except:
            print("Saving metadata file failed".format(ID))
    finally:
            pass